# Aprendizem Profunda - GRU
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [82]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [83]:
df = pd.read_csv("data/small_dataset.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [ ]:
def to_csv(results_df, name):
    os.makedirs("submissao2", exist_ok=True)
    path = f"submissao2/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\t\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [ ]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [86]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 4053, Unique texts: 4051
Overlap between train and test after deduplication: 0


### Tokenizer

In [87]:
# Tokenize the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=100)
X_val = pad_sequences(X_val_seq, maxlen=100)
X_test = pad_sequences(X_test_seq, maxlen=100)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [88]:
data_set_professor = pd.read_csv("data/dataset3_inputs.csv", sep=';', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=100)  

### Early Stopping

In [89]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=10,            
    restore_best_weights=True,           
)

### Reduce_lr

In [90]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## GRU

In [95]:
model_gru = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    GRU(32, dropout=0.3, recurrent_dropout=0.3),
    BatchNormalization(),
    Dense(16, activation="relu", kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation="sigmoid", kernel_regularizer=l2(0.01))
])

# Compile
optimizer = Adam(learning_rate=0.001)
model_gru.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Train
model_gru.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    shuffle=True
)

Epoch 1/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.5900 - loss: 0.8879 - val_accuracy: 0.8858 - val_loss: 0.7284 - learning_rate: 0.0010
Epoch 2/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.9033 - loss: 0.4274 - val_accuracy: 0.8935 - val_loss: 0.4383 - learning_rate: 0.0010
Epoch 3/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.9504 - loss: 0.2865 - val_accuracy: 0.9460 - val_loss: 0.2681 - learning_rate: 0.0010
Epoch 4/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.9632 - loss: 0.2294 - val_accuracy: 0.9614 - val_loss: 0.2013 - learning_rate: 0.0010
Epoch 5/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9621 - loss: 0.1831 - val_accuracy: 0.9522 - val_loss: 0.2074 - learning_rate: 0.0010
Epoch 6/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.9701 - loss: 0.1683 - val_accuracy: 0.9568 - val_loss: 0.2011 - learning_rate: 0.0010
Epoch 7/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.9831 - loss: 

# Correr Modelos

**Test Set**

In [ ]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_gru, X_test, test_ids, "gru")

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Predictions saved for dnn
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Predictions saved for simple_rnn
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Predictions saved for rnn
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Predictions saved for lstm
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Predictions saved for gru
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
              precision    recall  f1-score   support

         0.0       0.52      1.00      0.68       420
         1.0       0.00      0.00      0.00       391

    accuracy                           0.52       811
   macro avg       0.26      0.50      0.34       811
weighted avg       0.27      0.52      0.35       811



c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

**Avaliação**

In [ ]:
predict_and_save(model_gru, X_professor, data_set_professor["ID"], "gru_professor")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Predictions saved for dnn_professor
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Predictions saved for simple_rnn_professor
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Predictions saved for rnn_professor
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predictions saved for lstm_professor
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predictions saved for gru_professor


## Classification Reports

In [ ]:
print(classification_report(y_test, model_gru.predict(X_test) > 0.5))

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       420
         1.0       1.00      0.96      0.98       391

    accuracy                           0.98       811
   macro avg       0.98      0.98      0.98       811
weighted avg       0.98      0.98      0.98       811

